In [59]:
from pydantic import BaseModel, parse_obj_as, Field, constr
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn
from transformers import pipeline
import json
import re

In [95]:
# texts = ['Harry looked down in time to see a tiny, wrinkled, newborn bird poke its head out of the ashes. It was quite as ugly as the old one. "\'It\'s a shame you had to see him on a burning day,\' said Dumbledore, seating himself behind his desk. "\'He\'s really very handsome most of the time, wonderful red and gold plumage—facinating creatures, Phoenixes.', 
#          ' They can carry immensely heavy loads, their tears have healing powers, and they make highly faithful pets. In the shock of forks catching fire, Harry had forgotten what he was there for, but it all came back to him as Dumbledore settled himself in the high-backed chair behind the desk, and fixed Harry with his penetrating light blue stare.']

chunks = [
    {'timestamp': [0.0, 24.52], 
    'text': ' The dark figure streaming with fire raced towards them. The orcs yelled and poured over the stone gangways. Then Boromir raised his horn and blew. Loud the challenge rang and bellowed, like the shout of many throats under the cavernous roof. For a moment the orcs quailed and the fiery shadow halted. Then the echoes died as suddenly as a flame blown out by a dark wind, and the enemy advanced again.',
    'duration': 24.52}, 
    {'timestamp': [24.52, 47.64], 
    'text': 'Over the bridge! cried Gandalf, recalling his strength. ‘Fly! This is a foe beyond any of you. I must hold the narrow way. Fly!’ Aragorn and Boromir did not heed the command, but still held their ground, side by side, behind Gandalf at the far end of the bridge. The others halted just within the doorway at the halls end, and turned, unable to leave their leader to face the enemy alone.',
    'duration': 23.12
    }
    ]

In [99]:
class MusicGenInfo(BaseModel):    
    tone: str
    intensity: str
    setting: str
    tempo: str
    musical_instrument: str
    is_major_key: bool


In [100]:
class LLMPromptGenerator:
    def __init__(self, model_name="TheBloke/Llama-2-7b-Chat-GPTQ"): # device="cpu"
        self.model_name = model_name
        self.hf_pipeline = pipeline('text-generation', model=model_name, device_map='auto') # device=device)
        self.info = []
        self.long_term_prompt = None

        self.major_key = None
        self.instrument = None
        
        
    def generate_llm_prompt(self, text, prompt=None):
        
        if prompt is None:
            prompt = """
            TEXT: {} 
            
            TASK: In JSON Format, A piece music generated as background ambience for the above text would have these qualities:
            
        """
        
        #incorporate json format into prompt and return
        return prompt.format(text)
        
    def extract_json_from_llm_output(self, llm_output):
        pattern = r'\{.*?\}'  # Non-greedy match for JSON-like objects
        matches = re.findall(pattern, llm_output, re.DOTALL)

        if not matches:
            return {}  # Return an empty dict if no matches are found

        # Initialize a variable to keep track of the longest JSON match
        longest_match = ""
        for match in matches:
            # Update longest_match if the current match is longer
            if len(match) > len(longest_match):
                try:
                    # Check if the current match is a valid JSON
                    json.loads(match)
                    longest_match = match
                except json.JSONDecodeError:
                    # If not a valid JSON, continue to the next match
                    continue

        # Attempt to parse the longest match as JSON, if it exists
        if longest_match:
            try:
                json_obj = json.loads(longest_match)
                return json_obj
            except json.JSONDecodeError:
                # Should not happen, as we've already tested with json.loads
                return {}
        else:
            return {}  # Return an empty dict if no valid JSON objects were found


    
    def generate_musicgen_prompt(self, text, music_gen_info=MusicGenInfo, prompt=None, flush=False):
        # Generate prompt for the LLM
        llm_prompt = self.generate_llm_prompt(text, prompt=prompt)
        
        #prime the model to generate the proper json format
        parser = JsonSchemaParser(music_gen_info.schema())
        prefix_function = build_transformers_prefix_allowed_tokens_fn(self.hf_pipeline.tokenizer, parser)
        
        #get the output (in json format) from the LLM
        llm_output = self.hf_pipeline(llm_prompt, prefix_allowed_tokens_fn=prefix_function)
        generated_text = llm_output[0]['generated_text']
        
        #extract the json object from the generated_text and convert it to a dict.
        music_attributes = self.extract_json_from_llm_output(generated_text)
        
        #set/reset the long term attributes
        if flush or self.major_key is None: 
            self.major_key = music_attributes['is_major_key']
            self.instrument = music_attributes['musical_instrument']

        #create the prompt using fstring
        prompt = (f"Ambient Background music with a {music_attributes['tone']} tone and {music_attributes['intensity']} intensity, "
          f"using {self.instrument} instrumentation to create a {music_attributes['setting']} setting. "
          f"The piece moves at a {music_attributes['tempo']} pace, "
          f"{'in a major key' if self.major_key else 'in a minor key'}, "
          "evoking an immersive atmosphere.")
        
        return prompt
        
    def generate_from_chunks(self, chunks, music_gen_info=MusicGenInfo, prompt=None):
        prompts = []
        
        flush = False
        for chunk in chunks: 
            #some logic to determine when to flush could go here. 
            
            #generate the prompt and add it to the list
            prompts.append(self.generate_musicgen_prompt(chunk['text'], flush=flush))
            
            
        return prompts

In [101]:
extractor = LLMPromptGenerator()

/home/jupyter-chasewes/.conda/envs/mcmc/lib/python3.8/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [102]:
prompt = extractor.generate_musicgen_prompt(chunks[0]['text'])
prompt

/home/jupyter-chasewes/.conda/envs/mcmc/lib/python3.8/site-packages/transformers/generation/utils.py:1197: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'Ambient Background music with a dark tone and loud intensity, using orchestral to create a cave setting. The piece moves at a 100 pace, in a minor key, evoking an immersive atmosphere.'

In [94]:
all_prompts = extractor.generate_from_chunks(chunks)
for prompt in all_prompts:
    print(prompt)

/home/jupyter-chasewes/.conda/envs/mcmc/lib/python3.8/site-packages/transformers/generation/utils.py:1197: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Ambient Background music with a ominous tone and loud intensity, using dark_figure_streaming_with_fire_raced_towards_them to create a cavernous setting. The piece moves at a fast pace, in a minor key, evoking an immersive atmosphere.
Ambient Background music with a serious tone and soft intensity, using dark_figure_streaming_with_fire_raced_towards_them to create a outdoor setting. The piece moves at a moderate pace, in a minor key, evoking an immersive atmosphere.
